### 模型评估脚本

这个脚本用于评估基于transformers库的模型在古文翻译任务上的性能。

- **模型加载**: 加载预训练模型和分词器。
- **文本翻译**: 通过模型将文言文翻译为白话文。
- **评估过程**: 对模型的翻译结果进行BLEU评分，以评估翻译质量。

In [1]:
import json
import os
import time
import random
from typing import List, Dict

from transformers import AutoModel, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [2]:
# 定义全局常量
# model_name_or_path=模型路径

model_name_or_path = '/root/ChatGLM3/chatglm3-6b'

In [3]:
# 加载模型和分词器
model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True).half().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
model = model.eval()
# 打印模型的数据精度
print(f"model dtype: {model.dtype}")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

model dtype: torch.float16


In [4]:
# 提示构造
PROMPT_DICT = {
    "prompt_input": (
        "下面是一段文言文文本，请直接将它翻译成白话文。\n"
        "{terms}"  # 如有专业术语或特定背景，将在这里显示
        "#文言文文本:\n{input}\n"
        "#白话文文本:\n"
    )
}

# 功能函数
def generate_output_from_text(text_list: List[str]) -> List[Dict[str, str]]:
    """
    生成文本输出。

    :param text_list: 需要翻译的文言文文本列表。
    :param terms_list: 对应每个文言文文本的专业术语列表。每个列表包含词典，其中包含术语的源文本、标签和目标文本。
    :return: 包含模型翻译输出的字典列表。
    """
    result_list = []

    for idx, text in enumerate(text_list):
        # 使用PROMPT_DICT构造完整的输入提示
        prompt = PROMPT_DICT["prompt_input"].format(input=text, terms="")
        # 调用模型进行翻译
        result = model.chat(tokenizer, prompt, history=[])[0]
        result_list.append({"text": result})

    return result_list

In [5]:
# 使用方法：
text_list = ["老吾老，以及人之老；幼吾幼，以及人之幼"]
results = generate_output_from_text(text_list)
print(results[0]['text'])

要尊重老人，也要尊重别人的老人；要关心孩子，也要关心别人的孩子。


In [6]:
# 使用方法：
text_list = ["青，取之于蓝，而青于蓝；冰，水为之，而寒于水。"]
results = generate_output_from_text(text_list)
print(results[0]['text'])

青色是从蓝色中提取的，但青色比蓝色更青；冰是由水制成的，但冰的寒冷程度超过了水。


In [7]:
# 使用方法：
text_list = ["十三年，署武昌知府。吴三桂犯湖南，师方攻岳州，檄成龙造浮桥济师，甫成，山水发，桥圮，坐夺官。"]
results = generate_output_from_text(text_list)
print(results[0]['text'])

在十三年的时候，我担任武昌知府。吴三桂侵犯湖南，军队正准备攻击岳州，我发行了公文要求成龙建造浮桥以便军队渡河，但浮桥刚刚建成，山水爆发，桥梁受损，因此我失去了官职。


进行评估

In [8]:
# 定义全局常量
# eva_file_path=用于评估的文件路径
# output_file_path=评估结果输出路径

eva_file_path = r"/root/xm/HistoryTrans/data/version4/sampled_200_merged_output_20230812_190843.json"
output_file_path = "./data/eval_results/"

# 检查输出文件夹是否存在，如果不存在则创建
if not os.path.exists(output_file_path):
    os.makedirs(output_file_path)

# 读取评估数据
with open(eva_file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

In [9]:
def calculate_bleu(reference, candidate):
    reference = [list(reference)]
    candidate = list(candidate)
    return sentence_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1)

In [10]:
total_samples = len(data)

# 初始化评估结果
evaluation_results = {
    "scores": {"average_BLEU": 0},
    "infos": {
        "evaluation_time": time.strftime("%Y/%m/%d %H:%M:%S", time.localtime()),
        "model_name_or_path": model_name_or_path,
        "eva_file_path": eva_file_path,
        "total_samples": total_samples
    },
    "samples": []
}

# 开始评估
total_bleu_score = 0
print(f"开始评估，共 {total_samples} 个样本.")

for i, example in enumerate(data):
    inputs = example["input"]
    truths = [example["output"]]
    results = generate_output_from_text([inputs])
    results_str = results[0]['text']
    max_bleu_score = max(calculate_bleu(truth, results_str) for truth in truths)
    total_bleu_score += max_bleu_score

    # 将每个样本的评估结果添加到evaluation_results中
    evaluation_results["samples"].append({
        "inputs": inputs,
        "truth": truths[0],
        "results": results_str,
        "BLEU": max_bleu_score
    })

    # 打印进度和当前样本的BLEU分数
    print(f"评估进度: {i+1}/{total_samples}, 当前样本BLEU分数: {max_bleu_score:.2f}")

开始评估，共 200 个样本.
评估进度: 1/200, 当前样本BLEU分数: 0.13
评估进度: 2/200, 当前样本BLEU分数: 0.01
评估进度: 3/200, 当前样本BLEU分数: 0.16
评估进度: 4/200, 当前样本BLEU分数: 0.10
评估进度: 5/200, 当前样本BLEU分数: 0.28
评估进度: 6/200, 当前样本BLEU分数: 0.25
评估进度: 7/200, 当前样本BLEU分数: 0.54
评估进度: 8/200, 当前样本BLEU分数: 0.24
评估进度: 9/200, 当前样本BLEU分数: 0.01
评估进度: 10/200, 当前样本BLEU分数: 0.15
评估进度: 11/200, 当前样本BLEU分数: 0.04
评估进度: 12/200, 当前样本BLEU分数: 0.18
评估进度: 13/200, 当前样本BLEU分数: 0.11
评估进度: 14/200, 当前样本BLEU分数: 0.43
评估进度: 15/200, 当前样本BLEU分数: 0.18
评估进度: 16/200, 当前样本BLEU分数: 0.62
评估进度: 17/200, 当前样本BLEU分数: 0.26
评估进度: 18/200, 当前样本BLEU分数: 0.01
评估进度: 19/200, 当前样本BLEU分数: 0.06
评估进度: 20/200, 当前样本BLEU分数: 0.03
评估进度: 21/200, 当前样本BLEU分数: 0.07
评估进度: 22/200, 当前样本BLEU分数: 0.00
评估进度: 23/200, 当前样本BLEU分数: 0.03
评估进度: 24/200, 当前样本BLEU分数: 0.02
评估进度: 25/200, 当前样本BLEU分数: 0.03
评估进度: 26/200, 当前样本BLEU分数: 0.11
评估进度: 27/200, 当前样本BLEU分数: 0.17
评估进度: 28/200, 当前样本BLEU分数: 0.17
评估进度: 29/200, 当前样本BLEU分数: 0.22
评估进度: 30/200, 当前样本BLEU分数: 0.19
评估进度: 31/200, 当前样本BLEU分数: 0.27
评估进度: 32/200, 当前样本BLEU分数: 0.06
评

In [11]:
# 计算平均BLEU分数并保存评估结果
evaluation_results["scores"]["average_BLEU"] = total_bleu_score / total_samples
result_file = os.path.join(output_file_path, f"eval_resu_{time.strftime('%Y%m%d%H%M%S', time.localtime())}.json")
with open(result_file, "w", encoding="utf-8") as f:
    json.dump(evaluation_results, f, ensure_ascii=False, indent=4)

print(f"评估完成，平均BLEU分数为: {evaluation_results['scores']['average_BLEU']:.2f}")
print(f"评估结果已保存到 {result_file}")

评估完成，平均BLEU分数为: 0.16
评估结果已保存到 ./data/eval_results/eval_resu_20231217215841.json
